# Final Project: Fake News Detection

By Felix Daubner - Hochschule der Medien

Module 'Supervised and Unsupervised Learning' - Prof. Dr.-Ing. Johannes Maucher

## Data Access

As stated in [Problem Understanding](01_problem-understanding.ipynb) the training data is scraped from [POLITIFACT.com](https://www.politifact.com/). [POLITIFACT.com](https://www.politifact.com/) is a project run by non-profit organization Poynter Institue for Media Studies. Via its website POLITIFACT verifies the truth of political statements in the USA. It originated as an election-year project of the Tampa Bay Times. Since then it rates political statements into different truth categories. The statements originate from politicans, members of the US congress, White House and lobbyists. Every statement is reviewed and then classified into a category of either "true", "mostly true", "half true", "mostly false", "false", and "pants on fire". The rating into one of these classification is done by intensive research using the information which was public the day the statement was made. 

...

[2]

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Join Test Data from LIAR Dataset

The downloaded LIAR dataset should be used only as a validation dataset as described in [Problem Understanding](01_problem-understanding.ipynb). The LIAR dataset consists of three tsv-files which are already labeled as train, test and validation data. For the purposes of this project those three datasets should be joined to only one dataset which will then be used after training, evaluation and optimization of the fake-news and sentiment model to test the performance of both models.

Firstly, all three datasets are imported into a pandas DataFrame.

In [2]:
train = pd.read_csv("data/LIAR/train.tsv", header=None, sep="\t")

In [3]:
test = pd.read_csv("data/LIAR/test.tsv", header=None, sep="\t")

In [5]:
valid = pd.read_csv("data/LIAR/valid.tsv", header=None, sep="\t")

It is assumed that all datasets have the same number of columns and are ordered in the same way.
For control purposes, the shapes of all datasets are printed out as well as the sum of all rows of all three datasets. This number is used to later control whether the join of the datasets worked correctly. 

In [13]:
print(f"Shape train:\t{train.shape}")
print(f"Shape test:\t{test.shape}")
print(f"Shape valid:\t{valid.shape}")

print(f"Sum of rows:\t{train.shape[0]+test.shape[0]+valid.shape[0]}")

Shape train:	(10240, 14)
Shape test:	(1267, 14)
Shape valid:	(1284, 14)
Sum of rows:	12791


The three datasets are joined vertically using the 'concat' method.

In [7]:
LIAR = pd.concat([train, test, valid], axis=0)

The result is one DataFrame with a shape of 14 columns such as all the original datasets. The number of rows matches the sum of the rows from the earlier output. So we can be assume that the data was concatenated correctly.

In [9]:
LIAR.shape

(12791, 14)

Finally, the concatenated data is saved as csv-file.

In [ ]:
LIAR.to_csv("data/LIAR.csv", header=None, sep=";")

# Sources

[2] https://www.politifact.com/article/2018/feb/12/principles-truth-o-meter-politifacts-methodology-i/